In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\Project_Top2B\April_2023\regionwise\temporal_lobe\Healthy
data = pd.read_csv('Merged_Top2B_Temporal.csv')
data = data.drop(['Sample_ID'], axis = 1)
data.head(1)

Z:\Project_Top2B\April_2023\regionwise\temporal_lobe\Healthy


,Age,cg00018024,cg00018539,cg00019511,cg00029353,cg00030423,cg00034416,cg00034611,cg00041368,cg00048759,...,cg27569265,cg27597926,cg27601855,cg27624319,cg27629898,cg27633533,cg27635069,cg27658416,cg27658967,cg27664120
0,22.0,0.072326,0.849762,0.879962,0.079398,0.053969,0.05131,0.816422,0.88599,0.578528,...,0.168073,0.117982,0.169645,0.199086,0.641515,0.960283,0.402698,0.069964,0.072187,0.898755


In [3]:
# Replace '' values with NaN
data = data.replace('', np.nan)

# Loop over each column and replace NaN values with column means
for col in data.columns:
    # Convert the column to numeric data type
    data[col] = pd.to_numeric(data[col])
    
    # Calculate the mean of the column
    col_mean = data[col].mean()
    
    # Replace NaN values with the column mean
    data[col].fillna(col_mean, inplace=True)

data.to_csv("Merged_Top2B_Mean_tl.csv", index = False)
data.head(2)

,Age,cg00018024,cg00018539,cg00019511,cg00029353,cg00030423,cg00034416,cg00034611,cg00041368,cg00048759,...,cg27569265,cg27597926,cg27601855,cg27624319,cg27629898,cg27633533,cg27635069,cg27658416,cg27658967,cg27664120
0,22.0,0.072326,0.849762,0.879962,0.079398,0.053969,0.051310,0.816422,0.885990,0.578528,...,0.168073,0.117982,0.169645,0.199086,0.641515,0.960283,0.402698,0.069964,0.072187,0.898755
1,36.0,0.058923,0.852396,0.865666,0.063831,0.075482,0.062866,0.779562,0.883992,0.507011,...,0.164201,0.091953,0.162283,0.290112,0.687827,0.901325,0.321078,0.099531,0.085375,0.857039


In [4]:
X = data.drop(['Age'], axis = 1)
y = data['Age']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [5]:
rf_all_features = RandomForestRegressor(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

accuracy_score(y_test, rf_all_features.predict(X_test))

In [6]:
rfc = RandomForestRegressor(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestRegressor(n_estimators=142,
                                         random_state=RandomState(MT19937) at 0x26C01466640),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x26C01466640, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  69


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
selected_rf_features.to_csv ('CpG_BorutaPy_onAll_tl.csv', index = False)